In [4]:
import pandas as pd
from ydata_profiling import ProfileReport
from pyspark.sql import SparkSession

# Initialiser une session Spark
spark = SparkSession.builder \
    .appName("Nexialog Anomaly Detection") \
    .getOrCreate()



PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
file_path = (r"data/250203_tests_fixe_dns_sah_202412_202501.parquet") #dans .gitignore


In [3]:
donnee_brutes = pd.read_parquet(file_path, engine="pyarrow")  # ou engine="fastparquet"


In [4]:
donnee_brutes.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total
0,2024-12-01,01,old0,01_olt_1,01_peag_1,BU966,dsp_1,69_pebib_1,69_lyon,61,4.888439,0.834768,25,10.462500,0.843750,2.852823,0.550380,86
1,2024-12-01,01,old0,01_olt_2,01_peag_2,BU966,dsp_1,21_pebib_2,69_lyon,60,6.341362,2.033462,18,11.000000,0.346331,3.100490,0.451774,78
2,2024-12-01,01,M11,01_olt_3,01_peag_3,BU1464,dsp_1,69_pebib_3,69_lyon,23,6.845641,0.377309,9,11.437500,0.281250,3.258823,0.375156,32
3,2024-12-01,01,M11,01_olt_4,01_peag_3,BU1464,dsp_1,69_pebib_3,69_lyon,10,7.060350,0.392031,4,12.656250,2.079476,2.576471,1.272078,14
4,2024-12-01,01,M13,01_olt_5,01_peag_4,BU458,dsp_1,None,69_lyon,28,2.787241,0.410001,16,8.542969,0.512205,2.868750,0.714469,44


In [5]:
len(donnee_brutes)

9964847

In [16]:
donnee_brutes_réduite = donnee_brutes.sample(n=7000000, random_state=42)  # Prendre un échantillon


In [ ]:
profil = ProfileReport(donnee_brutes_réduite)  # explorative=True active toutes les analyses


: 

In [ ]:
profil.to_file("rapport_données_brutes.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/23 [01:14<?, ?it/s]
